In [1]:
import sinter
from typing import List 
import matplotlib.pyplot as plt
import rsmf 

formatter : plt.Formatter = rsmf.setup(r"\documentclass[a4paper,11pt,noarxiv]{quantumarticle}")

In [2]:
stats_memory : List[sinter.TaskStats] = sinter.stats_from_csv_files('./out/verification.csv')

In [49]:
fig : plt.figure = formatter.figure(wide=True)

subfigs = fig.subfigures(nrows=2, ncols=1)
axs = subfigs[0].subplots(nrows=1, ncols=2, sharex='all', sharey='row')
color_func = lambda _, curve_id: {'color': 'green'
                                    if curve_id == 'GaugeHoneycombCode'
                                    else 'red'}
subfigs[0].suptitle('Unbiased phenomenological noise,  $p= 0.001$, MWPM')

# Plot for phenomenological noise with varying d_x
sinter.plot_error_rate(
    ax=axs[0],
    stats=stats_memory,
    group_func=lambda stat: f"{stat.json_metadata['code_name']}",
    x_func=lambda stat: stat.json_metadata['d_x'],
    filter_func=lambda stat: (stat.json_metadata['h'] == 8 and 
                            stat.json_metadata['noise_model'] == 'phenomenological'),
    line_fits=('linear', 'linear'),
    plot_args_func=color_func
)

# Plot for phenomenological noise with varying h
sinter.plot_error_rate(
    ax=axs[1],
    stats=stats_memory,
    x_func=lambda stat: stat.json_metadata['h'],
    filter_func=lambda stat: (stat.json_metadata['d_x'] == 8 and 
                            stat.json_metadata['noise_model'] == 'phenomenological'),
    group_func=lambda stat: f"{stat.json_metadata['code_name']}",
    line_fits=('linear', 'linear'),
    plot_args_func=color_func
)

axs[0].set_xlim(0,25)
axs[0].set_ylim(0,1e-5)
axs[0].set_ylabel('Logical error rate')


handles, labels = axs[0].get_legend_handles_labels()
axs[0].legend(handles=[handles[0], handles[1]], labels=['$X^1Z^1$', '$X^1Y^1Z^1$'], loc='upper left')

axs = subfigs[1].subplots(nrows=1, ncols=2, sharex='all', sharey='row')

subfigs[1].suptitle('SD noise, $p = 0.0005$,  MWPM')

# Plot for circuit level noise with varying d_x
sinter.plot_error_rate(
    ax=axs[0],
    stats=stats_memory,
    group_func=lambda stat: f"{stat.json_metadata['code_name']}",
    x_func=lambda stat: stat.json_metadata['d_x'],
    filter_func=lambda stat: (stat.json_metadata['h'] == 8 and 
                            stat.json_metadata['noise_model'] == 'circuit_level_noise'),
    line_fits=('linear', 'linear'),
    plot_args_func=color_func
)

# Plot for circuit level noise with varying h
sinter.plot_error_rate(
    ax=axs[1],
    stats=stats_memory,
    x_func=lambda stat: stat.json_metadata['h'],
    filter_func=lambda stat: (stat.json_metadata['d_x'] == 8 and 
                            stat.json_metadata['noise_model'] == 'circuit_level_noise'),
    group_func=lambda stat: f"{stat.json_metadata['code_name']}",
    line_fits=('linear', 'linear'),
    plot_args_func=color_func
)


axs[0].set_xlim(0,25)
axs[0].set_ylim(0,1e-3)

axs[0].set_ylabel('Logical error rate')


axs[0].set_xlabel('$d_M$', labelpad=0)
axs[1].set_xlabel('$d_t$', labelpad=0)


axs[0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
x_s = [stat for stat in stats_memory if stat.json_metadata['code_name']]
fig.tight_layout()
plt.subplots_adjust(bottom=0.15, top=0.9, left=0.075, right=0.95)

plt.savefig(f"volume_verification.pdf")
plt.close()